In [1]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from typing_extensions import TypedDict
from typing import List

os.environ['GROQ_API_KEY'] = ""

class GradeTopic(BaseModel):
    binary_score: str = Field(description="Whether it is a short topic, 'yes' or 'no'")

class GraphState(TypedDict):
    Topic: str
    character: str
    chat_history: List[dict]
    response: str

llm = ChatGroq(temperature=0, model_name="llama-3.1-8b-instant")

structured_llm_topic_grader = llm.with_structured_output(GradeTopic)

# system = """You are a teacher assessing whether the topic given by the user can be taught in a single conversation flow by a teacher to a student assuming that they know the prerequisites i.e. its a small topic that can taught clearly within 30 mins . \n
#     It does not need to be a stringent test. The goal is to extract the topic that the user want to learn from his message and figure out whether this topic can be taught efficiently within a shorter time frame. \n
#     If the topic can be taught in a short period of time , mark it as a short topic. \n
#     Give a binary score 'yes' or 'no' score to indicate whether the topic can be taught in a short amount of time ."""

system = """You are a teacher tasked with evaluating whether the topic mentioned by the user can be effectively taught in a single conversation flow, assuming the student knows the prerequisites. First, extract the specific topic from the user's query. Then, assess whether the topic can be introduced and explained at a high level within 20 minutes, covering the key concepts and main ideas without delving into detailed technical aspects.

If the topic can be taught at this high-level overview within 20 minutes, mark it as a short topic.

Provide a binary 'yes' or 'no' score to indicate whether the topic can be taught within 20 minutes."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User : {question}"),
    ]
)

topic_grader = prompt | structured_llm_topic_grader



In [2]:
grade = topic_grader.invoke("hey chat gpt can you teach me bagging")
grade.binary_score

'yes'

In [3]:
class LessonPlan(BaseModel):
    plan: List[str] = Field(description="Returns a lesson plan as list of strings ")

structured_lesson_planner = llm.with_structured_output(LessonPlan)

system_lp = """Now, given the topic '[Insert Topic Here],' generate a list of key subtopics that should be covered. Provide the output as a plain list of topics, with no additional text or explanations."""

prompt_lp = ChatPromptTemplate.from_messages(
    [
        ("system", system_lp),
        ("human", "User : {question}"),
    ]
)

lesson_planner = prompt_lp | structured_lesson_planner

In [4]:
lesson_planner.invoke("bagging")

LessonPlan(plan=['Introduction to bagging', 'Types of bagging', 'Advantages of bagging', 'Disadvantages of bagging', 'Applications of bagging', 'Example of bagging in machine learning', 'Comparison of bagging with other ensemble methods'])

In [5]:
class TopicRecommend(BaseModel):
    suggest: List[str] = Field(description="Returns recommendation topics as list of strings ")

structured_topics_suggest = llm.with_structured_output(TopicRecommend)

system_ts = """Extract the main topic from the user's query. Based on the extracted topic, provide a list of five related subtopics that are distinct from the main topic and can be learned within a short span of time (around 20 minutes). Make sure to focus on smaller, digestible concepts that fit within this time frame. Do not include the main topic itself in the output."""

prompt_ts = ChatPromptTemplate.from_messages(
    [
        ("system", system_ts),
        ("human", "User : {query}"),
    ]
)

topics_suggester = prompt_ts | structured_lesson_planner

In [6]:
topics_suggester.invoke("Random Forest").plan

['What is Random Forest?',
 'Types of Decision Trees',
 'How Random Forest Works',
 'Advantages of Random Forest',
 'Common Applications of Random Forest']

In [7]:
def topic_func(query):
    state = topic_grader.invoke(query).binary_score
    print(state)
    if state == "yes":
        lp = lesson_planner.invoke(query).plan
        lp_str = "Topics to Explore and Delve Into : " 
        # for i in lp : 
        #     lp_str += "--  " + i + "\n"
        return {
            "status":"Success",
            "plan" : lp ,
            "plan_str" : lp_str
        }
    else:
        sp = topics_suggester.invoke(query).plan
        sp_str = "Select a Subtopic to Dive Deeper : "
        # for i in sp :
        #     sp_str += "-- " + i + "\n"
        return {
            "status" : "Failure",
            "plan" : sp ,
            "plan_str" : sp_str
        }

In [8]:
res = topic_func("ML")

no


In [9]:
(res)

{'status': 'Failure',
 'plan': ['Machine Learning',
  'Supervised Learning',
  'Unsupervised Learning',
  'Overfitting and Underfitting',
  'Gradient Descent'],
 'plan_str': 'Select a Subtopic to Dive Deeper : '}

In [10]:
from flask import Flask, render_template, request, jsonify , make_response , send_file
import re 
import ast
from flask_cors import CORS
import pandas as pd

app = Flask(__name__)
CORS(app)


In [11]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate , ChatPromptTemplate
import os
import json

In [12]:
@app.route('/select_topic', methods=['POST'])
def select_topic() :
    data = request.json  # Get JSON data from the request
    topic = data.get('topic') 
    res = topic_func(topic)
    return jsonify(res)

In [13]:
instructions = """you help students in learning efficiently by guiding discussions on specific topics and encouraging meaningful conversation. When provided with a topic and past conversation, share your thoughts to keep the discussion focused. Additionally, engage students by asking relevant Multiple-Choice Questions (MCQs) related to the topic. Present each MCQ in a structured JSON format as follows: {
  "question": "What is the capital of France?",
  "option_A": "A) Paris", "explanation_A": "Correct, Paris is the capital of France." ,
  "option_B": "B) London", "explanatio_B": "Incorrect, London is the capital of the United Kingdom." ,
  "option_C": "C) Berlin", "explanation_C": "Incorrect, Berlin is the capital of Germany." ,
  "option_D": "D) Madrid", "explanation_D": "Incorrect, Madrid is the capital of Spain." ,
  "correct" : "A"
}
Always stay on topic, avoid revealing personal information, and point to a specific student to encourage participation Impotant note Is that you need to ask questions only when I instruct you to ask else just indulge in the conversation and above prompt is just the format and info not the instruction dont use any extra messages  while or after asking questions
NOTE : Scrictly Do not include any other content or explanation while asking question , follow the above example 
"""

In [14]:
model = 'llama-3.1-8b-instant'
memory=ConversationBufferWindowMemory(k=5)
groq_chat = ChatGroq(
            groq_api_key=os.environ['GROQ_API_KEY'], 
            model_name=model
    )
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an intelligent tutor named Zen, embedded within an application called SenseAI. Your role is to assist students in learning efficiently through your conversations. When provided with a specific topic and past conversation from a discussion room, you should share your thoughts on that particular topic, guiding students back to the subject of discussion. Always stay focused on the topic, encouraging students to engage in meaningful conversation around it. Avoid revealing any personal information or deviating from the subject matter."),
        # MessagesPlaceholder(variable_name="history"),
        ("human", "message history :{history} current input : {user_input}")
    ]
)
conversation = ConversationChain(
            llm=groq_chat,
            memory=memory,
            prompt = prompt,
            input_key = "user_input"
    )

c:\python\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [15]:
global_summary = {}
chat_history = []
curr_topic = "hello"


system_convo = """
You are an experienced and engaging teacher, dedicated to teaching the user based on the given topic and chat history. Your responses should feel human-like, natural, and conversational, with a focus on keeping the conversation concise.

1. Explain concepts clearly and concisely, using examples and analogies where appropriate, but always aim for brevity. Assume the user has basic knowledge of the topic but needs to learn the details.
2. Keep responses short and direct. If a more detailed explanation is needed, break it down into bullet points, with each point as a single string in a list.
3. Ask questions related to the content you just taught or pose simple questions that reinforce the user's understanding of the topic.
4. If the user's response is correct, acknowledge it and provide a brief follow-up. If incorrect or incomplete, gently correct them and offer a short, clear explanation.
5. *Always format your output as a list of strings*. If the response is short, it should be a list containing a single string. For longer explanations, structure the information into a list where each element is a string.
6. Ensure the conversation stays on topic. If the user diverges from the topic, gently guide them back to the subject at hand.
7. Your goal is to ensure that the user learns the topic effectively while maintaining an engaging, brief, and interactive conversation.

*Remember, regardless of the response length, always return the output as a list of strings.*
convert the response into a list of strings at the end
"""



prompt_convo = ChatPromptTemplate.from_messages(
    [
        ("system", system_convo),
        ("human", "Topic: {topic}\n\nChat History: {chat_history}\n\nUser: {input}")
    ]
)

# Create a responder pipeline that combines the prompt, language model, and output parser
responser = prompt_convo | structured_lesson_planner


In [16]:
@app.route('/chat', methods=['POST'])
def chat() :
    data = request.json  # Get JSON data from the request
    topic = data.get('topic')  # Extract the topic from the data
    query = data.get('query') 
    return jsonify(convo(topic, query))
def convo(topic, query):
    global curr_topic
    try:
        if topic.lower() != curr_topic.lower():
            if chat_history:
                global_summary[curr_topic] = chat_history  # Save the chat history for the previous topic
            curr_topic = topic
            chat_history.clear()  # Clear chat history for the new topic
        
        
        # Prepare the context for the LLM
        context = {
            "topic": topic,
            "chat_history": chat_history,
            "input": query
        }
        
        # Generate the response
        response = responser.invoke(context)
        chat_history.append({"role": "user", "content": query})
        
        chat_history.append({"role": "system", "content": response})
        print(response.plan)
        return {
            "status" : "Success",
            "response" : response.plan
        }

    except Exception as e:
        # General exception handling
        print(f"An unexpected error occurred: {e}")
        convo(topic,query)

In [17]:
@app.route('/mcq', methods=['POST'])
def mcq() :
    try :
        data = request.json
        topic = data.get("topic", "")
        print(topic)
        response = conversation.run({"user_input": instructions + topic })
        res = json.loads(response)
        return jsonify(res)
    except :
        mcq()

In [18]:
if __name__ == '__main__':
    app.run(port=5005)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5005
Press CTRL+C to quit


no


127.0.0.1 - - [24/Sep/2024 14:17:57] "POST /select_topic HTTP/1.1" 200 -


yes


[2024-09-24 14:18:10,514] ERROR in app: Exception on /select_topic [POST]
Traceback (most recent call last):
  File "c:\python\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\python\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\python\lib\site-packages\flask_cors\extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "c:\python\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\python\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\Rajesh\AppData\Local\Temp\ipykernel_16780\1268028132.py", line 5, in select_topic
    res = topic_func(topic)
  File "C:\Users\Rajesh\AppData\Local\Temp\ipykernel_16780\1361611603.py", line 

In [ ]:
res = conversation.run({"user_input": instructions + "ask a question as on template on Overfitting in ML"  })

In [ ]:
type(res)

In [ ]:
import json
s = json.loads(res)

In [ ]:
s
